In [32]:
import pandas as pd
import os
import numpy as np
import re
import warnings
import math
warnings.filterwarnings('ignore') #supresses warnings for now
#warnings.filterwarnings(action='once') #shows warnings once

dataFilePath = os.path.join(os.getcwd(),'Data')

kicsSVdf = pd.read_csv(os.path.join(dataFilePath,'kics_structural_variations.txt'), sep = '\t', header = 0)
#kicsTEdf = pd.read_csv(os.path.join(dataFilePath,'kics_transposable_elements.txt'), sep = '\t', header = 0)
lfsSVdf = pd.read_csv(os.path.join(dataFilePath,'lfs_structural_variations.txt'), sep = '\t', header = 0)
#lfsTEdf = pd.read_csv(os.path.join(dataFilePath,'lfs_transposable_elements.txt'), sep = '\t', header = 0)
#kicsClinicdf = pd.read_csv(os.path.join(dataFilePath,'kics_clinical_edited.csv'), encoding='latin-1')
lfsClinicdf = pd.read_csv(os.path.join(dataFilePath,'lfs_clinical_main.tsv'), sep = '\t', header = 0)


In [33]:
kicsClinicdf = pd.read_csv(os.path.join(dataFilePath,'kics_clinical_edited.csv'), encoding='latin-1')
#print(kicsClinicdf.head(30))

In [34]:
kicsSVdf = kicsSVdf.loc[(kicsSVdf['AnnotSV type'] == 'full')]
lfsSVdf = lfsSVdf.loc[(lfsSVdf['AnnotSV type'] == 'full')]

In [35]:
"""

filter dfs

"""
def maxGDAFFilter(colName: str, df: pd.DataFrame)->pd.DataFrame:
    temp = df[colName].str.contains(pat=',')
    Index = list(np.where(temp == True)[0])
    temp = df[colName].iloc[Index]
    tempIndex = temp.index

    for i in tempIndex:
        tempList = temp[i].split(',')
        tempList = [float(a) for a in tempList]
        maxTemp = max(tempList)
        df[colName][i] = maxTemp

    df[colName] = df[colName].astype('float64')
    
    return df
    
kicsSVdf = maxGDAFFilter('GD_AF', kicsSVdf)
lfsSVdf = maxGDAFFilter('GD_AF', lfsSVdf)

In [36]:
listValSearch = ['291775', '298313', '315666', '320118', '18_1907']
# searched for kID of 63, 83, 156, 171, 219
# kID 120 141 and 232 do not have any matches...
#what would make this that much better is if it was a dict. Computationally less heavy?

for i in listValSearch: 
    kicsSVdf.drop(kicsSVdf[kicsSVdf['sample_id'].str.contains(i)==True].index, inplace = True)
    
kicsSVdf['SV chrom'] = kicsSVdf['SV chrom'].astype(str)
lfsSVdf['SV chrom'] = lfsSVdf['SV chrom'].astype(str)

kicsSVdf['sample_id'] = kicsSVdf['sample_id'].astype(str)
lfsSVdf['sample_id'] = lfsSVdf['sample_id'].astype(str)

kicsSVdf['sample_id'] = kicsSVdf['sample_id'].str.replace(" ", "")
lfsSVdf['sample_id'] = lfsSVdf['sample_id'].str.replace(" ", "")

listValSearch = ['63', '83', '156', '171', '219', '120', '141', '232']

kicsClinicdf['KiCS ID'] = kicsClinicdf['KiCS ID'].astype(str)
kicsClinicdf['KiCS ID'] = kicsClinicdf['KiCS ID'].str.replace(" ", "")

for i in listValSearch:
    kicsClinicdf.drop(kicsClinicdf[kicsClinicdf['KiCS ID']==i].index, inplace = True)
    
#kicsClinicdf.drop(kicsClinicdf.loc[kicsClinicdf['tissue_type']=='?'].index, inplace=True)
#kicsClinicdf.drop(kicsClinicdf.loc[pd.isna(kicsClinicdf['lfs_cancer_type_diagnosis'])].index, inplace=True)

In [37]:
# """
# Filter the ? out of of tissue type
# filter NAN from diagnosis

# """

# print(kicsClinicdf['lfs_cancer_type_diagnosis'].unique())
# list1temp = kicsClinicdf['lfs_cancer_type_diagnosis'].unique()
# #print(type(list1[1]), list1[1])

# kicsClinicdf.drop(kicsClinicdf.loc[kicsClinicdf['tissue_type']=='?'].index, inplace=True)
# kicsClinicdf.drop(kicsClinicdf.loc[kicsClinicdf['lfs_cancer_type_diagnosis']==list1temp[0]].index, inplace=True)
# print(kicsClinicdf['lfs_cancer_type_diagnosis'].unique())
# #print(kicsClinicdf['tissue_type'].value_counts())



In [38]:
def kIdAbbv(clinicDf, svDf, clinicCol, svCol):
    
    clinicDf.drop(clinicDf[clinicDf[clinicCol]=='Not applicable'].index, inplace = True)
    
    svDf[svCol] = svDf[svCol].astype(str)
    clinicDf[clinicCol] = clinicDf[clinicCol].astype(str)
    svDf[svCol] =  svDf[svCol].str.replace(" ", "")
    clinicDf[clinicCol] = clinicDf[clinicCol].str.replace(" ", "")
    
    list1 = svDf[svCol].unique()
    list2 = clinicDf[clinicCol].unique()
    
    svDict = dict()
    clinicDict = dict()
    
    for i in range (len(list1)):
        tempList = re.findall('\d{3,}', list1[i])

        #if empty, take the second one
        if not tempList:
            tempList = re.findall('\d+', list1[i])
            svDict[list1[i]] = tempList[1]

        #if not empty 
        else:
            svDict[list1[i]] = tempList[0]


    for i in range (len(list2)):    
        tempList = re.findall('\d{3,}', list2[i])
    
        #if empty, take the second one
        if not tempList:
            tempList = re.findall('\d+', list2[i])
            clinicDict[list2[i]] = tempList[1]

        #if not empty 
        else:
            clinicDict[list2[i]] = tempList[0]
    

    clinicDf['abbv_id'] = clinicDf[clinicCol].map(clinicDict)
    svDf['abbv_id'] = svDf[svCol].map(svDict)
    
    print(len(svDf[svCol].unique()))
    
    df = pd.merge(svDf[[svCol,'SV type','abbv_id']], 
                     clinicDf[[clinicCol, 'tumour_class','tissue_type','lfs_cancer_type_diagnosis', 'abbv_id']],
                     left_on = 'abbv_id', 
                     right_on = 'abbv_id', 
                     how='inner')

    return df

In [39]:
bigKDf = kIdAbbv(kicsClinicdf, kicsSVdf, 'CCP germline', 'sample_id')
print(len(bigKDf['abbv_id'].unique()))

#prints out how many we had before the id matching and how many we have left after the id matching

191
140


In [8]:
print(bigKDf)

        sample_id SV type abbv_id     CCP germline       tumour_class  \
0         18_5255     DUP    5255  18-5255-A-02-00       SOLID TUMORS   
1         18_5255     DUP    5255  18-5255-A-02-00       SOLID TUMORS   
2         18_5255     DUP    5255  18-5255-A-02-00       SOLID TUMORS   
3         18_5255     DUP    5255  18-5255-A-02-00       SOLID TUMORS   
4         18_5255     DUP    5255  18-5255-A-02-00       SOLID TUMORS   
...           ...     ...     ...              ...                ...   
5018938    292600     DEL  292600           292600  LEUKEMIA/LYMPHOMA   
5018939    292600     DUP  292600           292600  LEUKEMIA/LYMPHOMA   
5018940    292600     DUP  292600           292600  LEUKEMIA/LYMPHOMA   
5018941    292600     DEL  292600           292600  LEUKEMIA/LYMPHOMA   
5018942    292600     DEL  292600           292600  LEUKEMIA/LYMPHOMA   

        tissue_type lfs_cancer_type_diagnosis  
0                GI                       NaN  
1                GI        

In [9]:
"""Filter the ? and the nan"""

#print(bigKDf['tissue_type'].unique())
bigKDf.drop(bigKDf.loc[bigKDf['tissue_type']=='?'].index, inplace=True)
bigKDf.drop(bigKDf.loc[pd.isna(bigKDf['lfs_cancer_type_diagnosis'])].index, inplace=True)
print(len(bigKDf['abbv_id'].unique()))
#print(kicsClinicdf['tissue_type'].value_counts())

97


In [10]:
lfsSVdf['sample_id'] = lfsSVdf['sample_id'].astype(str)
lfsClinicdf['sample'] = lfsClinicdf['sample'].astype(str)

In [11]:
print(len(lfsSVdf['sample_id'].unique())) #only 65 to begin with D:
print(lfsSVdf['sample_id'].unique())

print(len(lfsClinicdf['sample'].unique()))

65
['4257' 'SJACT008_G' '808' '2085' '1087' '1092' '3805' '3136A' '4033'
 '2447' '353' '3806' '4854' '1478' '2262' '4028' '2957' '3688' '93' '1357'
 '2565' '4329' '4535_S1' '3298A' '2349' '2446' '3473' '5567' '3273' '3885'
 '3432' '1843' '3763' '3503' 'PD13489b' '2463' '2760A' '3643' 'SJACT001_G'
 '4227' '5537' '3367' 'SJACT002_G' '2815' '5536' '3634' '1476'
 'SJACT005_G' '3425' '3365' '2413' 'SJACT007_G' '3356' '3361' '3687_2'
 '3332' '94' '1355' '2784' '2465' '3319' 'PD7195b' '3334' '352' '2765']
460


In [12]:
print(lfsClinicdf.columns)

print(lfsSVdf.columns)

"""
drop unaffected? and NA? 
- Keeping unaffected and NA
Which column do i use for id
- sample
"""
#print(set(lfsSVdf['sample_id']).difference(set(lfsClinicdf['sample'])))

Index(['sample', 'meth_id', 'p53', 'tm_donor', 'tissue_type',
       'cancer_diagnosis', 'active_cancer', 'gender', 'cancer_atdraw',
       'cancer_atdraw_y5', 'cancer_atdraw_y1', 'firstcancer_atdraw',
       'age_diff_closest_cancer', 'systemic_treatment_atdraw', 'ageofonset',
       'agesamplecollection', 'cancer_num', 'cancer1_age_diff', 'cancer1',
       'cancer1_ageofonset', 'cancer2', 'cancer2_ageofonset', 'cancer3',
       'cancer3_ageofonset', 'cancer4', 'cancer4_ageofonset', 'cancer5',
       'cancer5_ageofonset', 'cancer6', 'cancer6_ageofonset', 'cancer7',
       'cancer7_ageofonset', 'cancer8', 'cancer8_ageofonset', 'protein',
       'family', 'mdm2', 'gdna.exon.intron', 'gdna.transcript', 'gdna.start',
       'gdna.end', 'gdna.genomic.notation', 'gdna.change', 'cdna.base.change',
       'cdna.codon', 'cdna.transcript', 'cdna.codon.notation',
       'protein.codon.change', 'protein.codon.num', 'Variant_Classification',
       'Variant_Type', 'alleles', 'MRN', 'EXT#', 'WGS', 

'\ndrop unaffected? and NA? \n- Keeping unaffected and NA\nWhich column do i use for id\n- sample\n'

In [13]:
"""
Can directly map the two
Apply on Filtered Prelim Graphs
"""

bigLDF = pd.merge(lfsSVdf[['sample_id','SV chrom','GD_AF']], 
                     lfsClinicdf[['sample','tissue_type', 'cancer_diagnosis', 'active_cancer', 'gender']],
                     left_on = 'sample_id', 
                     right_on = 'sample', 
                     how='inner')
print(bigLDF)
print(len(bigLDF['sample_id'].unique()))


        sample_id SV chrom     GD_AF sample tissue_type cancer_diagnosis  \
0            4257        1 -1.000000   4257      Breast           Breast   
1            4257        1  0.000047   4257      Breast           Breast   
2            4257        1 -1.000000   4257      Breast           Breast   
3            4257        1 -1.000000   4257      Breast           Breast   
4            4257        1  0.000098   4257      Breast           Breast   
...           ...      ...       ...    ...         ...              ...   
1434954      2765        X  0.000094   2765     Adrenal              ACC   
1434955      2765        X  0.000491   2765     Adrenal              ACC   
1434956      2765        X  0.000491   2765     Adrenal              ACC   
1434957      2765        X  0.000422   2765     Adrenal              ACC   
1434958      2765        Y -1.000000   2765     Adrenal              ACC   

        active_cancer gender  
0              Breast      F  
1              Breast    